## <center>Error for delayed SIR using EE and estimating from a fine grid solution </center>

DONE IN TWO NORM

In [1]:
from math import factorial
import numpy as np

## SIR model with delay
def SIR(beta, gamma):
    def SIR_dde(t, y1, y2):
        S, I, R = y1
        S_d, I_d, R_d = y2
        return np.array([-beta*S*I_d, beta*S*I_d - gamma*I, gamma*I])
    return SIR_dde


## History function
def phi_SIR(t):
    return np.array([0.8, 0.2, 0])  

## Linear interpolation for delay terms
def y_delayed(t, y, t_q):
    j = np.searchsorted(t, t_q)  
    j1, j2 = j-1, j     
    w = (t_q - t[j1]) / (t[j2] - t[j1])      
    y_delayed = (1 - w) * y[j1] + w * y[j2]
    return y_delayed

In [ ]:
## Explicit Euler
def EEsys_dde(f, phi, t_0, t_f, tau, N, history=False):

    ## Preliminaries
    h = (t_f - t_0) / N
    t = np.linspace(t_0, t_f, N+1)
    y0 = phi(t_0)
    y = np.zeros((len(t), len(y0)))
    y[0] = y0

    ## EE with linear interpolation for delay term
    for i in range(0, N):   
        if t[i] < tau:
            y[i+1] = y[i] + h*f(t[i], y[i], phi(t[i] - tau)) ## We have exact values for these delay terms, let's use them
        else:
            y[i+1] = y[i] + h*f(t[i], y[i], y_delayed(t, y, t[i] - tau)) ## We need to use linear interpolation for these delay terms

    if history == True: ## Wether the solution should include the history (values on the interval [t_0 - tau, t_0])
        t_hist = np.linspace(t_0-tau, t_0, int(N / (t_f // tau) + 1))
        y_hist = np.array([phi(ti) for ti in t_hist])
        t = np.concatenate((t_hist, t))
        y = np.concatenate((y_hist, y))

    return h, t, y


In [4]:
beta = 2
gamma = 1
t_0 = 0
t_f = 10
tau = 5

h, t, y = EEsys_dde(SIR(beta, gamma), phi_SIR, t_0, t_f, tau, 2**14, False)
finesol = y[:,-1]

N = 2**6
twonormresult = []
orderresult = []

for i in range(0,3):
    h, t, y = EEsys_dde(SIR(beta, gamma), phi_SIR, t_0, t_f, tau, N * 2 ** i, False)
    #This is the restriction to the actual coarser gride
    exact = finesol[0:2**14+1:2**(8-i)]  
    twonorm = np.sqrt(h) * np.linalg.norm(exact - y[:,-1])
    #np.linalg.norm(exact - y[:,-1],np.inf)
    twonormresult.append(twonorm)
    
errorvector = np.hstack(twonormresult)

## R(h) and p as log_2(R(h))
for i in range(0,len(errorvector)-1):
    order = np.log2(errorvector[i]/errorvector[i+1])
    orderresult.append(order)
    
print(orderresult)
print(errorvector)

[np.float64(1.0205948695896565), np.float64(1.0186789745319924)]
[0.02543293 0.01253622 0.00618748]
